In [1]:
import os 
os.chdir('..')

In [ ]:
# from app.data.lbox_handler import process_labelbox_data

# process_labelbox_data()

In [4]:
from app.inference import infer_model, get_labels_main, get_labels_tuned

from app.models import get_model, get_tokenizer


In [3]:
model_main = get_model()
tokenizer_main = get_tokenizer()

/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
infer_model(["My name is Noor", "I like the color Black Pyramids"],model_main, tokenizer_main, get_labels_main(), max_length=64)

[[{'entity': 'Noor', 'start_idx': 11, 'end_idx': 15, 'label': 'person'}]]

In [2]:
from app.models import get_tuning_tokenizer, get_tuning_model
from app.training.training_routine import get_trainer, prepare_data

model_tuned = get_tuning_model(path = "models/checkpoint-253-aug")
tokenizer_tuned = get_tuning_tokenizer()

/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
infer_model(["I like the 5 kgs", "I will go to the office tomorrow at 5 o'clock"], model_tuned, tokenizer_tuned, get_labels_tuned(), max_length=64)

[[{'entity': '5 ', 'start_idx': 11, 'end_idx': 13, 'label': 'Quantity'},
  {'entity': 'kgs', 'start_idx': 13, 'end_idx': 16, 'label': 'Units'}]]

In [6]:
from app.training.training_helpers import compute_metrics_2

train_set, test_set, _, _ = prepare_data(tokenizer_tuned, "merged.txt")
trainer = get_trainer(model_tuned, train_set, test_set, cuda=False, batch_size=2, compute_metrics=compute_metrics_2)

2024-08-18 03:01:16,510 - app.logger - WARNING - max_length is None, setting it to 64
2024-08-18 03:01:16,514 - app.logger - WARNING - max_length is None, setting it to 64


In [7]:
trainer.evaluate()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/187 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       B-Age       1.00      1.00      1.00        44
    B-Colors       0.96      1.00      0.98        52
  B-Currency       1.00      0.50      0.67         8
     B-Dates       1.00      0.97      0.99        70
    B-Prices       0.97      1.00      0.99       149
  B-Quantity       0.97      0.98      0.97       278
     B-Times       0.98      0.95      0.96        84
     B-Units       0.97      1.00      0.99       234
       I-Age       1.00      1.00      1.00       154
    I-Colors       0.88      0.94      0.91        32
  I-Currency       1.00      0.67      0.80         6
     I-Dates       1.00      0.97      0.99       220
    I-Prices       0.98      0.99      0.98       423
  I-Quantity       1.00      0.92      0.96        72
     I-Times       0.99      1.00      0.99       304
     I-Units       0.98      0.99      0.99       531
           O       1.00      0.99      0.99      3248

    accuracy              

{'eval_loss': 0.058077793568372726,
 'eval_done': 1,
 'eval_runtime': 14.8731,
 'eval_samples_per_second': 25.079,
 'eval_steps_per_second': 12.573}

In [7]:
from app.inference import infer

In [11]:
infer(["I will go to work at 5:00 AM and my car is red"], model_tuned=model_tuned, model_main=model_main, tokenizer_main=tokenizer_main, tokenizer_tuned=tokenizer_tuned, max_length=64)

{'main': [[{'entity': '5:00 ',
    'start_idx': 21,
    'end_idx': 26,
    'label': 'time'},
   {'entity': 'red', 'start_idx': 43, 'end_idx': 46, 'label': 'artwork'}]],
 'tuned': [[{'entity': '5', 'start_idx': 21, 'end_idx': 22, 'label': 'Times'},
   {'entity': ':00 AM ', 'start_idx': 22, 'end_idx': 29, 'label': 'Units'},
   {'entity': 'red', 'start_idx': 43, 'end_idx': 46, 'label': 'Colors'}]],
 'regex': [{'5:00 AM': {'start_idx': 21, 'end_idx': 28, 'entity': 'Time'},
   'red': {'start_idx': 43, 'end_idx': 46, 'entity': 'Color'}}]}

In [12]:
from app.regex_finders import regex_ner

In [13]:
regex_ner("I will go to work at 5:00 AM and my car is red at the speed of 50 km/hr")

{'5:00 AM': {'start_idx': 21, 'end_idx': 28, 'entity': 'Time'},
 'red': {'start_idx': 43, 'end_idx': 46, 'entity': 'Color'},
 'km': {'start_idx': 66, 'end_idx': 68, 'entity': 'Unit'}}

In [14]:
regex_ner("سأسافر 5 كيلومترات بسيارة حمراء في الساعة 5:00 صباحًا")

{'5:00 ': {'start_idx': 42, 'end_idx': 47, 'entity': 'Time'},
 ' حمراء': {'start_idx': 25, 'end_idx': 31, 'entity': 'Color'},
 'كيلومترات': {'start_idx': 9, 'end_idx': 18, 'entity': 'Unit'}}